In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [2]:
import sys

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append('./../src/')
from manuscript import sankey_side_by_side as sankey
from manuscript import clustering, datasets, inout, export

pd.options.display.max_columns = 200
mpl.rcParams["figure.figsize"] = (10, 8)
mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["font.family"] = "Arial"

import IPython.display
IPython.display.display(IPython.display.HTML("<style>.container { width:90% !important; }</style>"))

fonts = inout.get_resource_path('fonts')
for f in os.listdir(fonts):
    if f.endswith(".ttf"):
        mpl.font_manager.fontManager.addfont(f"{fonts}/{f}")

In [4]:
user = 'general'     # defines top hierarchy of output folder
outfolder = '04e_clustering_triangle'    # name of notebook
save = False

In [5]:
def dump_figure(name):
    if save:
        export.image(
            user,
            f'{outfolder}/{name}',
        )

In [6]:
def dump_table(df, name):
    if save:
        export.full_frame(
            user, 
            f'{outfolder}/{name}', 
            df, 
            index=True,
            date=True
        )

# Get Data, as in reference

In [7]:
data = pd.read_csv(
    inout.get_material_path('general/03_overwrite_PF_Cr/03data-external_220901_1010.csv.gz'), 
    index_col=0)

In [8]:
data.shape

(12495, 72)

In [9]:
data = data.reset_index()

List of columns for clustering

In [10]:
data_columns = clustering.get_reference_data_columns()

Get data that we will run clustering on

In [11]:
data_mtx_orig = data[data_columns].copy()

## 0. Preparation of shared aspects

Let's create groups of variables which share those high correlations. Let's try different cutoffs for high correlation, as they produce different results

In [12]:
cutoff_groups_on_orig = clustering.identify_related_features(data_mtx_orig)


In [13]:
data_mtx_as_pct = data_mtx_orig.rank(axis=0, pct=True)
data_dist_col = clustering.get_distances(
    data_mtx_as_pct.transpose(), approach='nan_euclidean')   
col_tree = clustering.get_tree(data_dist_col, approach='ward')

In [14]:
threshold_for_relatedness = 0.7
cutoff_groups = cutoff_groups_on_orig[threshold_for_relatedness]

In [15]:
approaches = {}

## 1. Similarity approach

In [16]:
data_mtx = data_mtx_orig.copy()
data_mtx = data_mtx.rank(axis=0, pct=True)

data_mtx_for_similarity = data_mtx.copy()
data_mtx_for_similarity = clustering.reweight_related_features(
    data_mtx_for_similarity, 
    approach='mean_rank', 
    groups=cutoff_groups)

In [17]:
corr_mtx = data_mtx_for_similarity.transpose().corr("pearson")
data_dist = clustering.get_distances(corr_mtx, approach='euclidean')   
tree = clustering.get_tree(df_dist=data_dist, approach='ward')

In [18]:
out, assignments = clustering.table_with_assignments(
    tree=tree,
    labels=data.index
)

In [19]:
sign_mortality_similarity = clustering.get_sign_mortalities(
    df_assigned_clusters=out,
    df_with_mortality=data
)
sign_mortality_similarity.loc[:, 'approach'] = 'similarity'

In [20]:
approaches = {
    'Similarity' : {
        'feature_matrix': data_mtx,   # data_mtx_orig.rank(axis=0, pct=True),
        'data_dist':data_dist,
        'tree':tree,
        'assignments_table':out,
        'assignments':assignments
    }
}

## 2. Rank-euclidean approach

In [21]:
data_mtx = data_mtx_orig.copy()
data_mtx = data_mtx.rank(axis=0, pct=True)

data_mtx = data_mtx.copy()
data_mtx = clustering.reweight_related_features(
    data_mtx, 
    approach='square_root', 
    groups=cutoff_groups)

In [22]:
rank_data_dist = clustering.get_distances(data_mtx, approach='nan_euclidean')
rank_tree = clustering.get_tree(rank_data_dist, approach='ward')
rank_out, rank_assignments = clustering.table_with_assignments(
    tree=rank_tree,
    labels=data.index
)

sign_mortality_rank = clustering.get_sign_mortalities(
    df_assigned_clusters=rank_out, 
    df_with_mortality=data)
sign_mortality_rank.loc[:, 'approach'] = 'rank-eucliden'

In [23]:
approaches['Ranked-Euclidean'] = {
        'feature_matrix':data_mtx,
        'data_dist':rank_data_dist,
        'tree':rank_tree,
        'assignments_table':rank_out,
        'assignments':rank_assignments
}

## 3. Normalized-euclidean approach 

In [24]:
data_mtx = data_mtx_orig.copy()

In [25]:
to_winsor_both = [
    'Temperature', 'Heart_rate', 'Systolic_blood_pressure',
    'Diastolic_blood_pressure', 'Mean_arterial_pressure',
    'Respiratory_rate', 'PEEP', 'Plateau_Pressure',
    'ABG_pH', 'ABG_PaCO2', 'ABG_PaO2', 'PaO2FIO2_ratio',
    'Hemoglobin', 'Bicarbonate', 'Albumin', 'LDH',
    'Lactic_acid'
]
to_winsor_right = [
    'Norepinephrine_rate', 'Urine_output',
    'Lung_Compliance', 'WBC_count', 'Lymphocytes',
    'Neutrophils', 'Platelets', 'Creatinine',
    'Bilirubin', 'CRP', 'D_dimer', 'Ferritin',
    'Procalcitonin'
]
to_winsor_left = [
    'Oxygen_saturation'
]


WINSOR_THRESHOLD_PCT = 1

for column in data_columns:
    col = data_mtx[column].dropna()
    lower = np.percentile(col, WINSOR_THRESHOLD_PCT)
    upper = np.percentile(col, 100 - WINSOR_THRESHOLD_PCT)
    if column in to_winsor_both or column in to_winsor_left:
        data_mtx.loc[data_mtx[column] < lower, column] = lower
    if column in to_winsor_both or column in to_winsor_right:
        data_mtx.loc[data_mtx[column] > upper, column] = upper
        
    

to_log2 = [
    'Norepinephrine_rate', 'ABG_PaO2', 
    'Creatinine', 'Bilirubin', 'D_dimer', 
    'Ferritin', 'LDH', 'Lactic_acid',
    'Procalcitonin'
]

for c in to_log2:
    data_mtx[c] = np.log2(data_mtx[c])

to_quantize = [
    'ECMO_flag', 'Intubation_flag', 'Hemodialysis_flag', 'CRRT_flag',
    'Norepinephrine_flag',
    'GCS_eye_opening', 'GCS_motor_response',
    'GCS_verbal_response', 'RASS_score', 'PEEP', 'FiO2',
    'PEEP_changes', 'Respiratory_rate_changes',
    'FiO2_changes'
]

for c in to_quantize:
    data_mtx[c] = data_mtx[c].rank(pct=True)

for c in list(set(data_columns) - set(to_quantize)):
    col = data_mtx[c]
    col = (col - np.nanmin(col)) / (np.nanmax(col) - np.nanmin(col))
    data_mtx[c] = col

In [26]:
data_mtx = clustering.reweight_related_features(
    data_mtx, 
    approach='square_root', 
    groups=cutoff_groups)



In [27]:
norm_data_dist = clustering.get_distances(data_mtx, approach='nan_euclidean')
norm_tree = clustering.get_tree(norm_data_dist, approach='ward')
norm_out, norm_assignments = clustering.table_with_assignments(
    tree=norm_tree,
    labels=data.index
)

In [28]:
sign_mortality_norm = clustering.get_sign_mortalities(
    df_assigned_clusters=norm_out, 
    df_with_mortality=data)
sign_mortality_norm.loc[:, 'approach'] = 'norm-euclidean'

In [29]:
approaches['Normalized-Euclidean']  = {
        'feature_matrix':data_mtx,
        'data_dist':norm_data_dist,
        'tree':norm_tree,
        'assignments_table':norm_out,
        'assignments':norm_assignments
    }

# 3. Test triangle inequality

In [30]:
approach = 'Normalized-Euclidean'

In [31]:
import random

In [32]:
%%time



for approach in approaches.keys():

    r = approaches[approach]['data_dist']
    r = pd.DataFrame(r)

    agg = []
    boot=10000000
    for n in range(boot):

        ch = random.choices(r.index,k=3)

        while len(set(ch))<3:
            ch = random.choices(r.index,k=3)

        lengths = sorted(
            [
                r.loc[ch[0], ch[1]],
                r.loc[ch[0], ch[2]],
                r.loc[ch[1], ch[2]]
            ])

        diff = (lengths[0] + lengths[1]) - lengths[2]

        agg.append({'diff': diff, 'points': ch})

    res = pd.DataFrame(agg)

    f = res['diff']>=0
    res.loc[:, 'valid'] = f

    print(approach, ':', sum(~f), 'out of', boot)

Similarity : 0 out of 10000000
Ranked-Euclidean : 262 out of 10000000
Normalized-Euclidean : 447 out of 10000000
CPU times: user 11min 5s, sys: 3min 16s, total: 14min 22s
Wall time: 2h 55min 31s
